# DFJ Model

In [4]:
import gurobipy as gp
from gurobipy import GRB
from itertools import combinations


def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # Retrieve solution
        vals = model.cbGetSolution(x)
        #print(vals)
        selected = [(i,j) for i,j in x.keys() if vals[i,j] > 0.5]
        
        print("selected: ", selected)
        # Find the shortest cycle in the selected edges
        tours = subtour(selected)
        for tour in tours:
            if len(tour) < n:
                # Add subtour elimination constraint
                print("tour: ", tour)
                edges_to_add = [(tour[i], tour[i+1]) for i in range(len(tour)-1)] + [(tour[-1], tour[0])]
                model.cbLazy(gp.quicksum(x[i,j] for i,j in edges_to_add) <= len(tour)-1)
            if len(tour) ==n:
                print("\nFeasible Tour: ", tour)
                print("Objective Value: ", model.cbGet(GRB.Callback.MIPSOL_OBJ), "\n")
        
import networkx as nx

def subtour(selected_edges):
    G = nx.DiGraph()
    G.add_edges_from(selected_edges)
    cycles = list(nx.simple_cycles(G))
    return cycles

In [67]:
def subtour(selected_edges):
    def dfs(node, start, visited, path):
        if node == start and path:
            # Canonicalize the tour by rotating it to start from the smallest node
            min_index = path.index(min(path))
            canonical_tour = path[min_index:] + path[:min_index]
            cycles.add(tuple(canonical_tour))
            return
        if node in visited:
            return
        visited.add(node)
        for _, neighbor in filter(lambda edge: edge[0] == node, selected_edges):
            dfs(neighbor, start, visited, path + [node])
        visited.remove(node)

    nodes = set()
    for i, j in selected_edges:
        nodes.add(i)
        nodes.add(j)

    visited = set()
    cycles = set()

    for node in nodes:
        dfs(node, node, visited, [])

    return [list(tour) for tour in cycles]


In [13]:
# Create a sample asymmetric distance matrix
n = 300  # Number of cities
dist = {
    (i, j): (i+j+1) if i != j else 0
    for i in range(n) for j in range(n) if i != j
}

# Generate random distances in the range 10 to 30
dist = {
    (i, j): random.randint(10, 30) if i != j else 0
    for i in range(n) for j in range(n) if i!= j
}

In [14]:






m = gp.Model()

# Create variables
x = m.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='x')

# Add constraint
m.addConstrs(x.sum(i, '*') == 1 for i in range(n))
m.addConstrs(x.sum('*', j) == 1 for j in range(n))

# Optimize model
m._vars = vars
m.Params.lazyConstraints = 1
m.optimize(subtourelim)

solution = m.getAttr('x', x)
selected = [(i,j) for i,j in solution if solution[i,j] > 0.5]
tour = subtour(selected)

print('\n\n')
print('Optimal tour: %s' % str(selected))
print('Tour in order: ', tour)
print('Optimal cost: %g' % m.objVal)


Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 600 rows, 89700 columns and 179400 nonzeros
Model fingerprint: 0x218fbfe9
Variable types: 0 continuous, 89700 integer (89700 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
selected:  [(0, 13), (1, 232), (2, 246), (3, 67), (4, 115), (5, 223), (6, 95), (7, 216), (8, 62), (9, 233), (10, 206), (11, 57), (12, 75), (13, 61), (14, 125), (15, 265), (16, 111), (17, 80), (18, 2), (19, 105), (20, 76), (21, 177), (22, 127), (23, 186), (24, 8), (25, 255), (26, 281), (27, 124), (28, 142), (29, 146), (30, 160), (31, 96), (32, 204), (33, 18), (34, 33), (35, 240), (36, 150), (37, 49), (38, 12), (39, 214), (40, 161), (41, 269), (42,

# MTZ Model

In [15]:
m = gp.Model()

# Create binary variables for edges
vars = m.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='e')

# Create continuous variables for MTZ
u = m.addVars(range(n), vtype=GRB.CONTINUOUS, name='u')

# Add degree constraints
m.addConstrs(vars.sum(i, '*') == 1 for i in range(n))
m.addConstrs(vars.sum('*', j) == 1 for j in range(n))

# Add MTZ constraints
for i in range(1, n):
    for j in range(1, n):
        if i != j:
            m.addConstr(u[i] - u[j] + n * vars[i, j] <= n - 1)

# Optimize model
m.optimize()

# Print solution
solution = m.getAttr('x', vars)
selected = [(i,j) for i,j in solution if solution[i,j] > 0.5]
print('Optimal tour:', selected)
print('Optimal cost:', m.objVal)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 89702 rows, 90000 columns and 446706 nonzeros
Model fingerprint: 0xa3c8fc91
Variable types: 300 continuous, 89700 integer (89700 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+01, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Presolve removed 0 rows and 1 columns
Presolve time: 1.44s
Presolved: 89702 rows, 89999 columns, 446706 nonzeros
Variable types: 299 continuous, 89700 integer (89700 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.27s

Solved with primal simplex

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   11253    3.0000000e+03   0.000000e+00   0.000000e+00      5s

Root relaxation:

In [18]:
import gurobipy as gp
from gurobipy import GRB
import random

# Sample asymmetric distance matrix
n =300
#n = 5  # Number of cities

# Generate random distances in the range 10 to 30
# dist = {
#     (i, j): random.randint(10, 30) if i != j else 0
#     for i in range(n) for j in range(n) if i != j
# }

m = gp.Model()

# Create binary variables for edges
vars = m.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='e')

# Create continuous variables for MTZ
u = m.addVars(range(n), vtype=GRB.CONTINUOUS, name='u')

# Add degree constraints
m.addConstrs(vars.sum(i, '*') == 1 for i in range(n))
m.addConstrs(vars.sum('*', j) == 1 for j in range(n))

# Add MTZ constraints
for i in range(1, n):
    for j in range(1, n):
        if i != j:
            m.addConstr(u[i] - u[j] + n * vars[i, j] <= n - 1)

# Provide a warm start by setting a random permutation for the u vector
random_permutation = random.sample(range(n), n)
for i, val in enumerate(random_permutation):
    u[i].start = val # warm start

# Optimize model
m.optimize()

# Print solution
solution = m.getAttr('x', vars)
selected = [(i,j) for i,j in solution if solution[i,j] > 0.5]
print('Optimal tour:', selected)
print('Optimal cost:', m.objVal)


Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 89702 rows, 90000 columns and 446706 nonzeros
Model fingerprint: 0xb909f774
Variable types: 300 continuous, 89700 integer (89700 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+01, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]

User MIP start produced solution with objective 5774 (0.49s)
Loaded user MIP start with objective 5774

Presolve removed 0 rows and 1 columns
Presolve time: 1.37s
Presolved: 89702 rows, 89999 columns, 446706 nonzeros
Variable types: 299 continuous, 89700 integer (89700 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.26s

Solved with primal simplex

Root relaxation: objective 3.000000e+03, 11253 iterations, 1.